### Grouping for users and predicting the users characteristics by text of the debates

In [26]:
from sklearn import metrics
from scipy.sparse import hstack
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from collections import defaultdict
from sklearn.metrics import roc_auc_score
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from collections import defaultdict
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, svm 
from sklearn.metrics import accuracy_score
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings("ignore")

In [74]:
text=pd.read_csv("non_tokenized.csv")

In [75]:
text.head()

,Unnamed: 0,text,user,side,category,title,url,ideology,party,gender,ethnicity,religion,win_ratio
0,0,Some context on the resolution. Normally I wou...,Mikal,Pro,Miscellaneous,Covenys recent behavior would be a net detrime...,Mikal,Right,Independent,Male,White,Agnostic,0.9684
1,1,We can dismiss most everything in his last rou...,Mikal,Pro,Miscellaneous,Covenys recent behavior would be a net detrime...,Mikal,Right,Independent,Male,White,Agnostic,0.9684
2,2,On to rebuttals1) IntroI'm not sure what he is...,Mikal,Pro,Miscellaneous,Covenys recent behavior would be a net detrime...,Mikal,Right,Independent,Male,White,Agnostic,0.9684
3,3,This is a debate challenge to Cassie on a foru...,Mikal,Con,Politics,Islam is a civilized religion,Mikal,Right,Independent,Male,White,Agnostic,0.9684
4,4,Resending this to Zaro as this was intended fo...,Mikal,Con,Politics,Islam is a civilized religion,Mikal,Right,Independent,Male,White,Agnostic,0.9684


In [81]:
per_users=text[["text","user","ideology","party","gender","ethnicity","religion"]]

In [91]:
per_users.dropna(inplace=True)
per_users=per_users.groupby(["user","ideology","party","gender","ethnicity","religion"],as_index=False)["text"].apply(lambda x: ' '.join(x))


In [92]:
per_users

,user,ideology,party,gender,ethnicity,religion,text
0,0905845,Other,Republican Party,Female,White,Christian,"alright, let's go!\n\r\nyou can go first!"
1,091915,Other,Independent,Female,White,Christian,"Hello, I want to start off with the fact that ..."
2,10438,Other,Independent,Female,White,Christian,"Pageants are performances, in every meaning of..."
3,105060,Other,Other,Male,White,Atheist,"I sincerely believe it is possible.I, personal..."
4,11BangBangEnvyme,Right,Democratic Party,Male,White,Christian,Obama is the best choice for president. Mitt R...
...,...,...,...,...,...,...,...
3961,zneuser93,Center,Democratic Party,Male,White,Christian,My band:\r\n1:MC: Tupac http://en.wikipedia.or...
3962,zobothehobo,Left,Democratic Party,Female,Middle Eastern,Agnostic,I don't believe texting in class is THAT bad. ...
3963,zoetwinn,Left,Democratic Party,Female,White,Atheist,First round is acceptance. Preferably my oppon...
3964,zorasbrown,Center,Democratic Party,Male,Asian,Christian,dear con . thers no sound without a listener ....


In [93]:
# Step - a : Remove blank rows if any.
per_users['text'].dropna(inplace=True)
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
per_users['text_final'] = [entry.lower() for entry in per_users['text']]
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
per_users['text_final']= [word_tokenize(entry) for entry in per_users['text_final']]
# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(per_users['text_final']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'title_final'
    per_users.loc[index,'text_final'] = str(Final_words)

Dataset has target variables with high imbalanced classes!! We have to solve this 

In [94]:
random.seed(500)

In [95]:
#dividing in train (70% of the dataset) and test set (30% of the dataset)
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(per_users['text_final'],per_users[['gender','religion','ethnicity','ideology']],test_size=0.3)

In [96]:
Encoder = LabelEncoder()

#word vectorization
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(per_users["text_final"])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)



In [97]:
#Label encode the target variable — 
#This is done to transform Categorical data of string type in the data set 
#into numerical values which the model can understand.

#gender
Train_Y_gender = Encoder.fit_transform(Train_Y['gender'])
Test_Y_gender = Encoder.fit_transform(Test_Y['gender'])

#religion
Train_Y_rel = Encoder.fit_transform(Train_Y['religion'])
Test_Y_rel = Encoder.fit_transform(Test_Y['religion'])

#ethnicity
Train_Y_ethn = Encoder.fit_transform(Train_Y['ethnicity'])
Test_Y_ethn = Encoder.fit_transform(Test_Y['ethnicity'])

#ideology
Train_Y_id = Encoder.fit_transform(Train_Y['ideology'])
Test_Y_id = Encoder.fit_transform(Test_Y['ideology'])

#stacking into a vector
y_train=np.stack((Train_Y_gender,Train_Y_rel,Train_Y_ethn,Train_Y_id),axis=1)
y_test=np.stack((Test_Y_gender,Test_Y_rel,Test_Y_ethn,Test_Y_id),axis=1)

In [98]:
#using we can use the argument class_weight='balanced'  
#to penalize mistakes on the minority class by an amount proportional to how under-represented it is.
SVM = svm.SVC(C=1.0, kernel='linear',class_weight='balanced', decision_function_shape='ovo',probability=True)
# Create the Multioutput Classifier
mor = MultiOutputClassifier(SVM)
# Train the regressor
mor.fit(Train_X_Tfidf, y_train)
# Generate predictions for testing data
y_pred = mor.predict(Test_X_Tfidf)



In [99]:
print("SVM Metrics for Gender")
print(f"The F1 score is {f1_score(y_test[:,0], y_pred[:,0], average='weighted')}")

SVM Metrics for Gender
The F1 score is 0.7501101015571209


In [100]:
# What about AUROC?
print(roc_auc_score(y_test[:,0],mor.predict_proba(Test_X_Tfidf)[0],  multi_class='ovo') )


0.619954156536508


In [101]:
print("SVM Metrics for Religion")
print(f"The F1 score is {f1_score(y_test[:,1], y_pred[:,1], average='weighted')}")

SVM Metrics for Religion
The F1 score is 0.39238020301186466


In [102]:
# What about AUROC?
print(roc_auc_score(y_test[:,1],mor.predict_proba(Test_X_Tfidf)[1],  multi_class='ovo') )


0.6169420860766472


In [103]:
print("SVM Metrics for Ethnicity")
print(f"The F1 score is {f1_score(y_test[:,2], y_pred[:,2], average='weighted')}")

SVM Metrics for Ethnicity
The F1 score is 0.4749311260058872


In [104]:
# What about AUROC?
print(roc_auc_score(y_test[:,2],mor.predict_proba(Test_X_Tfidf)[2],  multi_class='ovo') )


0.590607199036932


In [105]:
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score Ideology")
print(f"The F1 score is {f1_score(y_test[:,3], y_pred[:,3], average='weighted')}")

SVM Accuracy Score Ideology
The F1 score is 0.36210725611600775


In [106]:
# What about AUROC?
print(roc_auc_score(y_test[:,3],mor.predict_proba(Test_X_Tfidf)[3],  multi_class='ovo') )


0.5771528292711622
